# AsyncIO

In [1]:
import asyncio # Se importa la libreria asyncio

"""
Definimos las corutinas con la palabra clave async, 
las corutinas (en Javascript se les llama Promesas) 
son funciones de Python asíncronas que pueden 
suspenderse y reanudarse.
"""
async def hola(): # Definimos la corutina hola()
    """
    La corutina hola imprimira la palabra "Hola" 
    despues de un segundo de espera 
    """
    
    """
    await es una palabra clave de python que permite hacer que una corutina
    espere a que se complete una operacion asincrona antes de seguir con el 
    codigo. Y la funcion sleep() de la libreria asyncio permite pausar la corutina.
    Basicamente estamos diciendo espera a que sleep se termine de ejecutar
    """
    await asyncio.sleep(1) # Dormimos la corutina
    print("Hola") # Imprimimos

async def mundo():
    """
    La corutina mundo imprimira la palabra "Mundo" 
    despues de dos segundos de espera 
    """
    await asyncio.sleep(2)
    print("Mundo")

async def main():
    # asyncio.gather permite ejecutar varias corutinas de forma concurrente
    await asyncio.gather(hola(), mundo())

try:
    # Creamos un instancia de un bucle eventos
    bucle = asyncio.get_event_loop()
    bucle.run_until_complete(main())
    bucle.close()
except NotImplementedError:
    pass

Hola
Mundo


In [2]:
# No hace falta importar la libreria asyncio puesto que ya la hemos importado en la anterior celda de codigo

mensajes = [["Yo soy el Camino, la Verdad y la Vida. Nadie va al Padre sino por mí.", 2], 
            ["Si me conocéis a mí, conoceréis también a mi Padre; desde ahora lo conocéis y lo habéis visto.", 1], 
            ["Porque por tus palabras serás declarado justo y por tus palabras serás condenado.", 5], # Tenemos que tener cuidado con lo que decimos
            ["Todo el que mira a una mujer deseándola, ya cometió adulterio con ella en su corazón.", 1]]

async def menj1(mensaje: str = None, tiempo_espera: int = 1):
    await asyncio.sleep(tiempo_espera)
    print(mensaje) 

async def menj2(mensaje: str = None, tiempo_espera: int = 1):
    await asyncio.sleep(tiempo_espera)
    print(mensaje)

async def main():
    for mensaje in mensajes:
        await asyncio.gather(menj1(mensaje[0], mensaje[1]), menj2(mensaje[0], mensaje[1]))

try:
    bucle = asyncio.get_event_loop()
    bucle.run_until_complete(main())
    bucle.close()
except NotImplementedError:
    pass

Yo soy el Camino, la Verdad y la Vida. Nadie va al Padre sino por mí.
Yo soy el Camino, la Verdad y la Vida. Nadie va al Padre sino por mí.
Si me conocéis a mí, conoceréis también a mi Padre; desde ahora lo conocéis y lo habéis visto.
Si me conocéis a mí, conoceréis también a mi Padre; desde ahora lo conocéis y lo habéis visto.
Porque por tus palabras serás declarado justo y por tus palabras serás condenado.
Porque por tus palabras serás declarado justo y por tus palabras serás condenado.
Todo el que mira a una mujer deseándola, ya cometió adulterio con ella en su corazón.
Todo el que mira a una mujer deseándola, ya cometió adulterio con ella en su corazón.


In [ ]:
# https://docs.python.org/es/3/library/asyncio-queue.html#examples
# https://docs.python.org/3/library/asyncio-task.html#coroutines
# https://docs.python.org/3/reference/compound_stmts.html#the-async-for-statement

async def poner(cola: asyncio.Queue):
    for i in range(1, 10 + 1):
        await cola.put(i) # Sirve para agregar un valor a cola

async def sacar(cola: asyncio.Queue):
    for i in range(cola.qsize() + 1): # qsize() devuelve la longitud de la cola
        valor = await cola.get() # get() sirve para recuperar el primer valor de la cola y lo elimina
        print(valor)

async def main():
    mi_cola = asyncio.Queue() # Creamos una Cola
    tarea1 = asyncio.create_task(poner(mi_cola)) # Creamos una tarea para la corutina poner y la guardamos
    tarea2 = asyncio.create_task(sacar(mi_cola)) # Creamos una tarea para la corutina sacar y la guardamos
    await tarea1 # Ejecutamos la tarea con la corutina poner
    await tarea2 # Ejecutamos la tarea con la corutina sacar

await main()

1
2
3
4
5
6
7
8
9
10


In [7]:
# https://superfastpython.com/asyncio-async-for/#Example_of_async_for_with_Asynchronous_Generator

async def iterar(iterador: iter):
    # print("Hola") # Curiosamente si realizamos instrucciones no asincronas funciona
    for valor in iterador:
        await asyncio.sleep(1)
        yield valor

async def main():
    lista = [5,2,7,8,9,2,1]
    async for valor in iterar(lista):
        print(valor)

await main()

5
2
7
8
9
2
1


In [32]:
import asyncio

class contador:
    def __init__(self, valor) -> None:
        self.final = valor
        self.inicial = 0

    def __aiter__(self):
        return self

    async def __anext__(self):

        valor_inicial = self.inicial
        
        if valor_inicial == self.final:
            # print("Hola") # La condicion se ejecuta justo antes de terminar la ejecucion de la clase
            raise StopAsyncIteration
            
        self.inicial += 1
            
        return self.inicial

async def main():
    async for i in contador(10):
        print(i)

if __name__ == "__main__":
    await main()

1
2
3
4
5
6
7
8
9
10


# Multithreading

In [16]:
from threading import Thread

def contar(n):
    for num in range(1, n + 1):
        print(num)

hilo_1 = Thread(target=contar, args=(7,))
hilo_2 = Thread(target=contar, args=(5,))

hilo_1.run()
hilo_2.run()

1
2
3
4
5
6
7
1
2
3
4
5


In [8]:
import threading

cadena = "Porque tres son los que dan testimonio en el cielo: el Padre, el Verbo y el Espíritu Santo; y estos tres son uno."
vocales = ['a', 'e', 'i', 'o', 'u', 'á', 'é', 'í', 'ó', 'ú']
vocales_encontradas = dict()

def procesar(cadena: str):
    num_vocales = 0
    cadena = cadena.lower()
    
    for caracter in cadena:
        if caracter in vocales:
            num_vocales += 1
            vocales_encontradas[caracter] = 0
            vocales.remove(caracter)

    return num_vocales

def ocurrencias(cadena: str, caracter: str):
    vocales_encontradas[caracter] = cadena.count(caracter)

num_hilos = procesar(cadena)

for num_hilo in range(num_hilos):
    # https://es.stackoverflow.com/questions/549738/como-accedo-al-valor-de-una-key-de-un-diccionario-python
    hilo = threading.Thread(target=ocurrencias, args=(cadena, list(vocales_encontradas.keys())[num_hilo]))
    if 'get_ipython' in globals(): # Esta condicional sirve para saber si estamos en un entorno de Jupyter Notebook
        hilo.run()
    else: # Si estamos en otro entorno que no sea Jupyter Notebook, ejecutaremos el hilo de esta otra forma
        temp.start()
        
print(vocales_encontradas)

{'o': 11, 'u': 4, 'e': 14, 'a': 3, 'i': 4, 'í': 1}


In [9]:
import time
from threading import Thread

class temporizador(Thread): # Hereramos de la clase Thread
    
    def __init__(self, segundos: int, nombre: str) -> None:
        super().__init__() # Llamamos al construtor de la clase Thread
        self.tiempo = segundos
        self.nombre = nombre
        
    def run(self): # Sobreescribimos el metodo run
        for _ in range(self.tiempo):
            time.sleep(1)
            print(f"El nombre del temporizador es {self.nombre}, el tiempo asignado es de {self.tiempo} segundos.")

        print("Se ha finalizado la ejecucion correctamente.")

if __name__ == "__main__":
    temp = temporizador(7, "Vida")

    if 'get_ipython' in globals():
        temp.run()
    else:
        temp.start()

El nombre del temporizador es Vida, el tiempo asignado es de 7 segundos.
El nombre del temporizador es Vida, el tiempo asignado es de 7 segundos.
El nombre del temporizador es Vida, el tiempo asignado es de 7 segundos.
El nombre del temporizador es Vida, el tiempo asignado es de 7 segundos.
El nombre del temporizador es Vida, el tiempo asignado es de 7 segundos.
El nombre del temporizador es Vida, el tiempo asignado es de 7 segundos.
El nombre del temporizador es Vida, el tiempo asignado es de 7 segundos.
Se ha finalizado la ejecucion correctamente.


In [15]:
import threading

mi_bloqueo = threading.Lock()
global contador # global es una palabra clave de Python que nos permite declarar una variable global
contador = 0

def incrementar(bloqueo: threading.Lock):
    global contador # Aqui llamamos a la variable global declarada anteriormente
    bloqueo.acquire() # Con acquire bloqueamos el hilo bloqueamos, es decir, las posteriores ejecuciones del hilo
    contador += 1
    print(contador)
    # bloqueo.release() # Con release desbloqueamos el hilo, es decir, permitimos que el hilo vuelva a ejecutarse

try:
    hilo = Thread(target=incrementar, args=(mi_bloqueo,))
    for _ in range(5):
        hilo.run()
except AttributeError: # Manejamos las excepcion que surge al intentar ejecutar 2 veces el hilo
    pass

1


In [7]:
import queue
import threading

mensajes = queue.Queue()

def encolar(mensaje: str, cola: queue.Queue):
    cola.put(mensaje)

def desencolar(cola: queue.Queue):
    mensaje = cola.get()
    print(mensaje)

hilos = list()
hilo_1 = threading.Thread(target=encolar, args=(str(7),mensajes))
hilo_1.run()
hilo_2 = threading.Thread(target=desencolar, args=(mensajes,))
hilo_2.run()
hilo_3 = threading.Thread(target=encolar, args=(str(5),mensajes))
hilo_3.run()
hilo_4 = threading.Thread(target=encolar, args=(str(4),mensajes))
hilo_4.run()
hilo_5 = threading.Thread(target=desencolar, args=(mensajes,))
hilo_5.run()

7
5
